# Справка

API Cross Web осуществляет расчет данных проекта Cross Web по измерению аудитории медиа объектов в интернете: проекты "Измерение интернет-аудитории" и "Измерение аудитории и профиля рекламных кампаний в интернете".
 
Обращаться к API Cross Web будем с помощью Jupyter Notebook, для этого необходимо владеть некоторыми терминами. 

Ниже описаны основные из них.

### usetype - тип пользования интернетом

Возможные варианты:

- 1 - Web Desktop
- 2 - Web Mobile
- 3 - App Mobile

## Типы расчетов


### Audience - расчет объема аудитории интернет-ресурсов

Доступны следующие статистики:

- adatg -  Average Daily Attention Of Group
- adatt -  Average Daily Attention
- addPerP  - Average Daily Duration на население
- addPerU - Average Daily Duration на пользователей
- adf - Average Daily Frequency
- ado - Average Daily OTS
- adr - Average Daily Reach
- adrPer - Average Daily Reach %
- affinity
- affinityAdr
- affinityAdrInternet
- affinityInternet
- atg - Attention Of Group
- att - Attention
- averageAge - Average Age
- awr - Average Weekly Reach
- awrPer - Average Weekly Reach %    
- dr - Days Reached
- drfd - Days Reached Frequency Distribution
- durationPerP - Duration на население
- durationPerU - Duration на пользователей
- exclusiveReach - Exclusive Reach
- exclusiveReachUt - Exclusive Reach Ut
- frequency
- grp - Gross Rating Point
- ots
- reach
- reachAdd - Reach Average Daily Duration
- reachN - Reach N+ Distribution
- reachPer - Reach %
- uni - Universe

### Duplication - расчет пересечения аудитории интернет-ресурсов

Доступны следующие статистики:

- adr - Average Daily Reach
- adrPer - Average Daily Reach %
- awr - Average Weekly Reach
- awrPer - Average Weekly Reach %    
- reach
- reachPer - Reach %
- uni - Universe

### Advertisement - расчет данных в рамках проекта "Измерение аудитории и профиля рекламных кампаний в интернете"

Доступны следующие статистики:

- ado - Average Daily OTS
- awo - Average Weekly OTS
- exclusiveReach - Exclusive Reach
- exclusiveReachUt - Exclusive Reach Ut
- frequency
- ots
- reach
- reachN - Reach N+ Distribution
- reachPer - Reach %
- uni - Universe

### Profile duplication - расчет пересечения аудитории в рамках проекта "Измерение аудитории и профиля рекламных кампаний в интернете"

Доступны следующие статистики:

- adr - Average Daily Reach
- adrPer - Average Daily Reach %
- awr - Average Weekly Reach
- awrPer - Average Weekly Reach %    
- reach
- reachPer - Reach %
- uni - Universe

### Audience media&profile - расчет совокупной аудитории интернет-ресурсов (Сross Web) и проекта "Измерение аудитории и профиля рекламных кампаний в интернете"

Доступны следующие статистики:

- exclusiveReach - Exclusive Reach
- frequency
- mediaExclusiveReach - Media Exclusive Reach
- ots
- profileExclusiveReach - Profile Exclusive Reach
- reach
- reachN - Reach N+ Distribution
- reachPer - Reach %
- uni - Universe

# Библиотека Mediascope API
Библиотека Mediascope содержит набор классов и методов, которые позволяют упростить работу с API.

Импортируйте библиотеку Mediascope API



In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.crossweb import catalogs as cwc
from mediascope_api.crossweb import tasks as cwt

Создайте объекты для работы с API

In [ ]:
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

# Работа с Mediascope Jupyter Notebook

В работе с Mediascope Jupyter Notebook можно выделить два основных действия:

- расчет заданий (создание задания, расчет, получение результата);
- работа со справочниками.

## Расчет заданий

Стандартный сценарий работы с заданиями:

- инициализация - импорт библиотеки и создание объектов для работы с API Cross Web;
- формирование задания;
- отправление задания на расчет и ожидание результата;
- получение результата и его преобразование в pandas.DataFrame;
- при необходимости сохранение результата в Excel.

При формировании задания можно указать параметры, приведенные ниже. 

- **filters** - фильтры нескольких типов:
    - **date_filter** - фильтр по периодам
    - **usetype_filter** - фильтр по типам пользования интернетом
    - **media_usetype_filter** - фильтр по типам пользования интернетом для медиа при работе с ноутбуком "audience_media&profile"
    - **profile_usetype_filter** - фильтр по типам пользования интернетом для рекламы при работе с ноутбуком "audience_media&profile"
    - **geo_filter** - фильтр по географии
    - **demo_filter** - фильтр по соц.-дему
    - **mart_filter** - фильтр по медиа-объектам
    - **duplication_mart_filter** - фильтр по пересечению (для типа расчета Duplication)
- **slices** - срезы
- **statistics** - статистики
- **scales** - шкалы, если заданы статистики со шкалами (drfd, reachN)

Из перечисленных параметров обязательными являются:

- **date_filter** - фильтр по периоду
- **usetype_filter** - фильтр по типам пользования интернетом (или **media_usetype_filter** и **profile_usetype_filter** при работе с ноутбуком "audience_media&profile")
- **scales** - шкалы, если заданы статистики со шкалами (drfd, reachN)
- **mart_filter и duplication_mart_filter** - фильтр по медиа-объектам и фильтр по пересечению для типа расчета Duplication

Без их указания рассчитать задание невозможно.

Для расчета заданий существуют методы в модуле mediascope_api.crossweb.tasks: 

- **build_task** - формирует задание: принимает указанные параметры, проверяет их и создает JSON для API Cross Web. Есть три типа заданий: _'media'_ (для расчета по медиа объектам), _'total'_ (для расчета Total Internet), _'ad'_ (для расчета по рекламным кампаниям); 
- **build_task_media_duplication** - формирует задание для расчета типа Duplication: принимает указанные параметры, проверяет их и создает JSON для API Cross Web. Тип заданий: _'media-duplication'_; 
- **send_task** - отправляет задание на расчет;
- **wait_task** - ожидает расчет;
- **get_result** - получает результат;
- **result2table** - преобразует результат из JSON в pandas.DataFrame.

**При формировании задания необходимо корректно задавать атрибуты для параметров.** 

Для **media** доступные атрибуты и допустимые для них значения можно получить методом `get_media_unit()`:

In [ ]:
cats.get_media_unit()

Для **media total** доступные атрибуты и допустимые для них значения можно получить методом `get_media_total_unit()`:

In [ ]:
cats.get_media_total_unit()

Для **duplication media** доступные атрибуты и допустимые для них значения можно получить методом `get_media_duplication_unit()`:

In [ ]:
cats.get_media_duplication_unit()

Для **рекламы** доступные атрибуты и допустимые для них значения можно получить методом `get_ad_unit()`:

In [ ]:
cats.get_ad_unit()

Для **duplication рекламы** доступные атрибуты и допустимые для них значения можно получить методом `get_media_duplication_unit()`:

In [ ]:
cats.get_profile_duplication_unit()

Для **media & рекламы** доступные атрибуты и допустимые для них значения можно получить методом `get_media_profile_unit()`:

In [ ]:
cats.get_media_profile_unit()

Создавать выражения для фильтров **geo_filter**, **demo_filter**, **mart_filter** и **duplication_mart_filter** можно с помощью следующих операторов:

    = (равно)
    != (не равно)
    > (больше)
    < (меньше)
    >= (больше или равно) 
    <= (меньше или равно)
    AND (И)
    OR (ИЛИ)
    IN() (вхождение в список значений)

## Вывод справки

Все методы библиотеки Mediascope имеют справку.

Для вывода справки нужно нажать комбинацию клавиш __Shift+Tab__
```
mtask.build_task(
```
или воспользоваться командой __help__

In [ ]:
help(mtask.build_task)

## Работа со справочниками

Для работы со справочниками в библиотеке Mediascope содержатся методы в модуле `mediascope_api.crossweb.catalogs`.

В ноутбуке [catalogs](catalogs.ipynb) приведены примеры получения данных из всех доступных справочников.